In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras.datasets import mnist
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm_notebook

from util import load_data, create_dataset

# Deep Autoencoder Structure I: Rumelhardt, Log2(N)

In [2]:
(x_train, y_train), (x_test, y_test) = load_data()

In [3]:
def create_model():
    #Sigmoid and ReLU are appropriate activation functions when having nonnegative data.
    model = keras.Sequential([
        layers.Input( shape=(784) ),
        layers.Dense(32, activation='relu' ),
        # 10 because: Rumelheadt proposed to use log2(N), Log2(784) approx 10.
        layers.Dense(10, activation='relu' ),
        layers.Dense(32, activation='relu' ),
        layers.Dense(784, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

def fit_model(model, x_train, y_train, x_test, y_test, normal=4, verbose=0):
    callbacks = [
        keras.callbacks.EarlyStopping(patience=2)
    ]
    train = x_train[y_train==normal]
    test  = x_test[y_test==normal]
    history = model.fit(train, train,
                    epochs=50,
                    batch_size=256,
                    verbose=verbose,
                    shuffle=True,
                    validation_data=(test, test),
                    callbacks=callbacks)
    return model

In [4]:
evals = np.zeros((10, 30))
for i in range(10):
    # Evaluate for all numbers
    for j in tqdm_notebook(range(30)):
        # Evaluate each method 30 times
        model = create_model()
        model = fit_model(model, x_train, y_train, x_test, y_test, normal=i)

        x, y, labels = create_dataset(x_test, y_test, normal=i)
        xhat = model.predict(x)
        err  = np.sum(np.abs(x-xhat), axis=1)
        # Max-Min normalize the error
        err /= np.max(err)
        # Compute AUC
        AUC = roc_auc_score(labels, err)
        evals[i,j] = AUC

print(np.mean(evals, axis=1))
print(np.std(evals, axis=1))


[0.98894037 0.99928536 0.85449153 0.91720125 0.94718365 0.93562822
 0.97437913 0.96887789 0.84675369 0.95629434]
[0.00384485 0.00026231 0.02362429 0.01638212 0.01397101 0.01201317
 0.0063243  0.00800612 0.02947517 0.00871649]


In [6]:
print(np.mean(evals, axis=1)*100)
print(np.std(evals, axis=1)*100)

[98.89403721 99.92853647 85.44915275 91.72012548 94.71836491 93.56282179
 97.43791319 96.88778915 84.67536912 95.6294341 ]
[0.38448477 0.02623144 2.36242881 1.63821203 1.39710101 1.20131739
 0.63243038 0.80061163 2.94751674 0.87164862]


# Deep Autoencoder Structure II: Tensorflow

In [10]:
def create_model():
    #Sigmoid and ReLU are appropriate activation functions when having nonnegative data.
    model = keras.Sequential([
        layers.Input( shape=(784) ),
        layers.Dense(128, activation='relu' ),
        layers.Dense(64, activation='relu' ),
        layers.Dense(32, activation='relu' ),
        layers.Dense(64, activation='relu' ),
        layers.Dense(128, activation='relu' ),
        layers.Dense(784, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

In [11]:
evals = np.zeros((10, 30))
for i in range(10):
    # Evaluate for all numbers
    for j in tqdm_notebook(range(30)):
        # Evaluate each method 30 times
        model = create_model()
        model = fit_model(model, x_train, y_train, x_test, y_test, normal=i)

        x, y, labels = create_dataset(x_test, y_test, normal=i)
        xhat = model.predict(x)
        err  = np.sum(np.abs(x-xhat), axis=1)
        # Max-Min normalize the error
        err /= np.max(err)
        # Compute AUC
        AUC = roc_auc_score(labels, err)
        evals[i,j] = AUC

print(np.mean(evals, axis=1))
print(np.std(evals, axis=1))


[0.99247015 0.99940516 0.89304437 0.94021795 0.96015553 0.95958437
 0.98497557 0.96598416 0.85261807 0.96915843]
[0.00276834 0.000143   0.0173217  0.00945851 0.00887324 0.0101099
 0.00334434 0.00808757 0.02397572 0.00611901]


In [12]:
print(np.mean(evals, axis=1)*100)
print(np.std(evals, axis=1)*100)

[99.24701513 99.94051575 89.30443749 94.02179525 96.01555268 95.95843714
 98.49755712 96.59841588 85.26180698 96.91584322]
[0.27683404 0.01430027 1.73217044 0.94585106 0.88732448 1.01099004
 0.33443403 0.80875728 2.39757166 0.61190133]
